# Hidden symmetry 
## The motivation
* Go beyond the integeralbility 
* Exploring the new structure of non-plannar amplitude
* Understanding the gauge/gravity duality deeply 

## The expected properties of the symmetry 
* Leave the amplitude invariant 
* The hidden symmetry is a global symmetry
* There are some factors which related to the ordering in the kinematic algebra when symmetry act on the external legs 
* The structure constant of the algebra should be the BCJ numerator 
* It should be Yangian symmetry for the planar amplitude
* It have martix representation in the spinor space

# The function of the algebra product  

In [2]:
 push!(LOAD_PATH, "/Users/gangchen/Dropbox/YM_BCJ/HiddenSymmetry");

In [3]:
using vectorDot
using SymEngine
using Latexify
using LaTeXStrings

In [4]:
p1=vecSym("p₁");p2=vecSym("p₂");p3=vecSym("p₃");p4=vecSym("p₄");p5=vecSym("p₅");p6=vecSym("p₆");
p7=vecSym("p₇");
ϵ1=vecSym("ϵ₁");
ϵ2=vecSym("ϵ₂");ϵ3=vecSym("ϵ₃");ϵ4=vecSym("ϵ₄");ϵ5=vecSym("ϵ₅");ϵ6=vecSym("ϵ₆");ϵ7=vecSym("ϵ₇");
a1=symbols("a_1");
a2=symbols("a_2");
a3=symbols("a_3");

In [5]:
mutable struct J <: Number
    comp
    index::Int64 
    mom
    function J(x,i::Int64,p)
        new(x,i,p)
    end
end

In [6]:
 struct scalarJTimes <: Number
        scalar::Basic
        vec::J
        function scalarJTimes(x::Basic,y::J)
            new(x,y)
        end
    end

In [7]:
function Base.show(io::IO, y::scalarJTimes)
        print(io,"[",y.scalar,"]"*" ",y.vec)
    end

In [27]:
  struct JAdd <: Number
        list
        function JAdd(x)
            new(x)
        end
    end

    function Base.show(io::IO, y::JAdd)
        str="";
        args=y.list;
        for i in 1:length(args)
            arg = args[i]
            str = string(str, arg)
            i != length(args) && (str *= " "*" "*"+"*" "*" ")
        end
        print(io, str)
    end

In [28]:
 import Base.*
    *(x::Basic,y::J)= scalarJTimes(x,y)
    *(x::Basic,y::scalarJTimes)= scalarJTimes(x*y.scalar,y.vec)
    *(y::scalarJTimes,x::Basic)=*(x::Basic,y::scalarJTimes)
    *(x::Basic,y::JAdd)= JAdd(x.*y.list)
    import Base.+
    +(x::J,y::J)= JAdd((x,y))
    #+(x::J,y...)= JAdd((x,y...))
    +(x::scalarJTimes,y::J)= JAdd((x,y))
    +(x::scalarJTimes,y::scalarJTimes)= JAdd((x,y))
   # +(x::scalarJTimes,y...)= JAdd((x,y...))
    +(x::JAdd,y::scalarJTimes)= JAdd(((x.list)...,y))
   +(y::scalarJTimes,x::JAdd)= +(x,y)
    +(x::JAdd,y::J)= JAdd(((x.list)...,y))
    +(y::J, x::JAdd)= JAdd((y, (x.list)...))
    +(y::JAdd, x::JAdd)= JAdd(((y.list)..., (x.list)...))
    #+(x::Tuple{J})= JAdd(x)

+ (generic function with 182 methods)

In [29]:
jj1=J((ϵ1),0,p1+p2) 

J(ϵ₁, 0, +(p₁, p₂))

In [30]:
jj1.comp.name[1]=='ϵ'

true

In [31]:
(dot(p1,ϵ2)*J((ϵ1),0,p1+p2)+dot(p1,ϵ2)*J((ϵ1),0,p1+p2) )+dot(p1,ϵ2)*J((ϵ1),0,p1+p2)

[(p₁⋅ϵ₂)] J(ϵ₁, 0, +(p₁, p₂))  +  [(p₁⋅ϵ₂)] J(ϵ₁, 0, +(p₁, p₂))  +  [(p₁⋅ϵ₂)] J(ϵ₁, 0, +(p₁, p₂))

In [32]:
function ⋆(j1::J,j2::J)
    if j1.comp.name[1]=='ϵ'
       # print("this is epsilon",j1.comp.name[1])
        dot(j2.comp,j1.mom)*J(j1.comp,0,j1.mom+j2.mom) +1//2*dot(j1.comp,j2.comp)*J(j2.mom,0,j1.mom+j2.mom)
    elseif j1.comp.name[1]=='p'
       # print("this is p")
        dot(j2.comp,j1.mom)*J(j1.comp,0,j1.mom+j2.mom) +dot(j1.comp,j2.comp)*J(j2.mom,0,j1.mom+j2.mom)
    end
end

    function ⋆(x::scalarJTimes,y::J)
        x.scalar*⋆(x.vec,y)
    end
    function ⋆(x::JAdd,y::J)
        xx = x.list;
        s=⋆(xx[1],y);
        for i=2:length(xx)
           s+=⋆(xx[i],y)
        end
        return s
    end
    function ⋆(x::JAdd,y::scalarJTimes)
       y.scalar*⋆(x,y.vec)
    end

⋆ (generic function with 4 methods)

In [33]:
ttt=J((ϵ1),0,p1)⋆J((ϵ2),0,p2)⋆J((ϵ3),0,p3)

[(p₁⋅ϵ₂)*((p₁⋅ϵ₃) + (p₂⋅ϵ₃))] J(ϵ₁, 0, +(p₁, p₂, p₃))  +  [(1/2)*(p₁⋅ϵ₂)*(ϵ₁⋅ϵ₃)] J(p₃, 0, +(p₁, p₂, p₃))  +  [(1/2)*((p₁⋅ϵ₃) + (p₂⋅ϵ₃))*(ϵ₁⋅ϵ₂)] J(p₂, 0, +(p₁, p₂, p₃))  +  [(1/2)*(p₂⋅ϵ₃)*(ϵ₁⋅ϵ₂)] J(p₃, 0, +(p₁, p₂, p₃))

In [34]:
latexify(ttt)

L"$[(p₁⋅ϵ₂)*((p₁⋅ϵ₃) + (p₂⋅ϵ₃))] J(ϵ₁, 0, +(p₁, p₂, p₃))  +  [(1/2)*(p₁⋅ϵ₂)*(ϵ₁⋅ϵ₃)] J(p₃, 0, +(p₁, p₂, p₃))  +  [(1/2)*((p₁⋅ϵ₃) + (p₂⋅ϵ₃))*(ϵ₁⋅ϵ₂)] J(p₂, 0, +(p₁, p₂, p₃))  +  [(1/2)*(p₂⋅ϵ₃)*(ϵ₁⋅ϵ₂)] J(p₃, 0, +(p₁, p₂, p₃))$"

# BCJ relations and the map from DDM basis to minimal basis

In [20]:
expand((a1+a2+3a3)^10)

10*a_2*a_1^9 + 196830*a_2*a_3^9 + 45*a_2^2*a_1^8 + 295245*a_2^2*a_3^8 + 120*a_2^3*a_1^7 + 262440*a_2^3*a_3^7 + 210*a_2^4*a_1^6 + 153090*a_2^4*a_3^6 + 252*a_2^5*a_1^5 + 61236*a_2^5*a_3^5 + 210*a_2^6*a_1^4 + 17010*a_2^6*a_3^4 + 120*a_2^7*a_1^3 + 3240*a_2^7*a_3^3 + 45*a_2^8*a_1^2 + 405*a_2^8*a_3^2 + 10*a_2^9*a_1 + 30*a_2^9*a_3 + 30*a_3*a_1^9 + 405*a_3^2*a_1^8 + 3240*a_3^3*a_1^7 + 17010*a_3^4*a_1^6 + 61236*a_3^5*a_1^5 + 153090*a_3^6*a_1^4 + 262440*a_3^7*a_1^3 + 295245*a_3^8*a_1^2 + 196830*a_3^9*a_1 + 270*a_2*a_3*a_1^8 + 3240*a_2*a_3^2*a_1^7 + 22680*a_2*a_3^3*a_1^6 + 102060*a_2*a_3^4*a_1^5 + 306180*a_2*a_3^5*a_1^4 + 612360*a_2*a_3^6*a_1^3 + 787320*a_2*a_3^7*a_1^2 + 590490*a_2*a_3^8*a_1 + 1080*a_2^2*a_3*a_1^7 + 11340*a_2^2*a_3^2*a_1^6 + 68040*a_2^2*a_3^3*a_1^5 + 255150*a_2^2*a_3^4*a_1^4 + 612360*a_2^2*a_3^5*a_1^3 + 918540*a_2^2*a_3^6*a_1^2 + 787320*a_2^2*a_3^7*a_1 + 2520*a_2^3*a_3*a_1^6 + 22680*a_2^3*a_3^2*a_1^5 + 113400*a_2^3*a_3^3*a_1^4 + 340200*a_2^3*a_3^4*a_1^3 + 612360*a_2^3*a_3^5*a_1^2

In [21]:
tt=expand((dot(p1,ϵ1)+dot(p2,ϵ2))^10)

10*(p₁⋅ϵ₁)*(p₂⋅ϵ₂)^9 + 45*(p₁⋅ϵ₁)^2*(p₂⋅ϵ₂)^8 + 120*(p₁⋅ϵ₁)^3*(p₂⋅ϵ₂)^7 + 210*(p₁⋅ϵ₁)^4*(p₂⋅ϵ₂)^6 + 252*(p₁⋅ϵ₁)^5*(p₂⋅ϵ₂)^5 + 210*(p₁⋅ϵ₁)^6*(p₂⋅ϵ₂)^4 + 120*(p₁⋅ϵ₁)^7*(p₂⋅ϵ₂)^3 + 45*(p₁⋅ϵ₁)^8*(p₂⋅ϵ₂)^2 + 10*(p₁⋅ϵ₁)^9*(p₂⋅ϵ₂) + (p₁⋅ϵ₁)^10 + (p₂⋅ϵ₂)^10

In [22]:
nice(tt)

L"$(p_2\cdot\epsilon_2)^{10} + 120  (p_1\cdot\epsilon_1)^{3}  (p_2\cdot\epsilon_2)^{7} + 45  (p_1\cdot\epsilon_1)^{2}  (p_2\cdot\epsilon_2)^{8} + 210  (p_1\cdot\epsilon_1)^{4}  (p_2\cdot\epsilon_2)^{6} + 252  (p_1\cdot\epsilon_1)^{5}  (p_2\cdot\epsilon_2)^{5} + 10  (p_1\cdot\epsilon_1)  (p_2\cdot\epsilon_2)^{9} + 210  (p_1\cdot\epsilon_1)^{6}  (p_2\cdot\epsilon_2)^{4} + 120  (p_1\cdot\epsilon_1)^{7}  (p_2\cdot\epsilon_2)^{3} + 45  (p_1\cdot\epsilon_1)^{8}  (p_2\cdot\epsilon_2)^{2} + 10  (p_1\cdot\epsilon_1)^{9}  (p_2\cdot\epsilon_2) + (p_1\cdot\epsilon_1)^{10}$"

# Ansatz of the symmetry action on the external legs

# Equations from the requirement of the amplitude invariant.